In [ ]:
!pip install torch-geometric

In [ ]:
!pip install torch-sparse

In [ ]:
!pip install torch-scatter

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
from torch_geometric.data import DataLoader

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/yoochoose-clicks.dat', header=None)
df.columns=['session_id','timestamp','item_id','category']

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
buy_df = pd.read_csv('/content/drive/MyDrive/data/yoochoose-buys.dat', header=None)
buy_df.columns=['session_id','timestamp','item_id','price','quantity']

In [ ]:
df.nunique()
buy_df.nunique()

session_id     509696
timestamp     1136477
item_id         19949
price             735
quantity           28
dtype: int64

In [ ]:
sampled_session_id = np.random.choice(df.session_id.unique(), 50000, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()
df.isna().sum()
from sklearn.preprocessing import LabelEncoder
def preprocess(dataframe,col):
    encoder = LabelEncoder()
    dataframe[col] = encoder.fit_transform(dataframe[col])
preprocess(df,"item_id")

In [ ]:
df['label'] = df.session_id.isin(buy_df.session_id)
df.head()

,session_id,timestamp,item_id,category,label
820,256,2014-04-07T14:16:36.706Z,11437,0,False
821,256,2014-04-07T14:17:57.517Z,11437,0,False
822,256,2014-04-07T14:18:25.346Z,10533,0,False
823,256,2014-04-07T14:18:34.007Z,11415,0,False
1328,468,2014-04-01T19:40:28.301Z,11424,0,False


In [ ]:
class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseBinaryDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return ['/content/drive/MyDrive/data/yoochoose_click_binary_1M_sess.dataset']

    def download(self):
        pass
    
    def process(self):
        
        data_list = []

        # process by session_id
        grouped = df.groupby('session_id')
        for session_id, group in tqdm(grouped):
            sess_item_id = LabelEncoder().fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id
            node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id']].sort_values('sess_item_id').item_id.drop_duplicates().values

            node_features = torch.LongTensor(node_features).unsqueeze(1)
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]

            edge_index = torch.tensor([source_nodes,
                                   target_nodes], dtype=torch.long)
            x = node_features

            y = torch.FloatTensor([group.label.values[0]])

            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])
        

In [ ]:
dataset = YooChooseBinaryDataset(root='')

In [ ]:
##Shuffling and splitting the dataset
dataset = dataset.shuffle()
train_dataset = dataset[:30000]
val_dataset = dataset[30000:40000]
test_dataset = dataset[40000:]
len(train_dataset), len(val_dataset), len(test_dataset)

(30000, 10000, 10000)

In [ ]:
#loading the data using loader function
batch_size= 1024
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]


        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

In [ ]:
#create NN
embed_dim = 128
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.conv4 = SAGEConv(128, 128)
        self.pool4 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)        

        x = F.relu(self.conv1(x, edge_index))

        x, edge_index, _, batch, _,_ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _,_ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _,_ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv4(x, edge_index))

        x, edge_index, _, batch, _,_ = self.pool4(x, edge_index, None, batch)
        x4 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3 +x4

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x

In [ ]:
#view data
for data in train_dataset:
    print(data)
    break

Data(x=[3, 1], edge_index=[2, 2], y=[1])


In [ ]:
device = torch.device('cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()

In [ ]:
##train network
def train():
    model.train()
    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)

In [ ]:
from sklearn.metrics import roc_auc_score
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)
    
    return roc_auc_score(labels, predictions)

In [ ]:
for epoch in range(20):
    loss = train()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)    
    test_acc = evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))

Epoch: 000, Loss: 0.06037, Train Auc: 0.99351, Val Auc: 0.61986, Test Auc: 0.61250
Epoch: 001, Loss: 0.05380, Train Auc: 0.99378, Val Auc: 0.63745, Test Auc: 0.63070
Epoch: 002, Loss: 0.04834, Train Auc: 0.99139, Val Auc: 0.64678, Test Auc: 0.64062
Epoch: 003, Loss: 0.05368, Train Auc: 0.99276, Val Auc: 0.63718, Test Auc: 0.63110
Epoch: 004, Loss: 0.05693, Train Auc: 0.98596, Val Auc: 0.64484, Test Auc: 0.63699
Epoch: 005, Loss: 0.05614, Train Auc: 0.99115, Val Auc: 0.63681, Test Auc: 0.63186
Epoch: 006, Loss: 0.06619, Train Auc: 0.99248, Val Auc: 0.62115, Test Auc: 0.61789
Epoch: 007, Loss: 0.07584, Train Auc: 0.99210, Val Auc: 0.59790, Test Auc: 0.59246
Epoch: 008, Loss: 0.06502, Train Auc: 0.99394, Val Auc: 0.59316, Test Auc: 0.58873
Epoch: 009, Loss: 0.04729, Train Auc: 0.99608, Val Auc: 0.61764, Test Auc: 0.61189
Epoch: 010, Loss: 0.03502, Train Auc: 0.99676, Val Auc: 0.60956, Test Auc: 0.60600
Epoch: 011, Loss: 0.03269, Train Auc: 0.99685, Val Auc: 0.61260, Test Auc: 0.60829
Epoc